### Serial Tree Creation, Parallel Forest Creation

Let's begin with a small sample.

In [61]:
sample=[(1,[1,1,0,1]),
         (0,[0,0,1,1]),
         (1,[1,1,0,2]),
         (0,[1,1,0,1]),
         (0,[0,0,1,0]),
         (2,[1,0,2,1]),
         (2,[2,0,1,1]),
         (2,[1,2,1,1]),
         (0,[0,1,1,1]),
         (1,[1,2,0,2]),
         (0,[1,1,1,1]),
         (0,[0,1,1,0]),
         (2,[2,2,2,1]),
         (2,[2,1,1,1]),
         (2,[2,2,1,1])]

The code below was borrowed from and then adapted for our array of labeledPoints from: http://www-users.cs.umn.edu/~kumar/dmbook/ch4.pdf

In [65]:
"""
This module holds functions that are responsible for creating a new
decision tree and for using the tree for data classificiation.
"""

def majority_value(data):
    """
    Creates a list of all values in the target attribute for each record
    in the data list object, and returns the value that appears in this list
    the most frequently.
    """
    return most_frequent([record[0] for record in data])

def most_frequent(lst):
    """
    Returns the item that appears most frequently in the given list.
    """
    lst = lst[:]
    highest_freq = 0
    most_freq = None

    for val in unique(lst):
        if lst.count(val) > highest_freq:
            most_freq = val
            highest_freq = lst.count(val)
            
    return most_freq

def unique(lst):
    """
    Returns a list made up of the unique values found in lst.  i.e., it
    removes the redundant values in lst.
    """
    lst = lst[:]
    unique_lst = []

    # Cycle through the list and add each value to the unique list only once.
    for item in lst:
        if unique_lst.count(item) <= 0:
            unique_lst.append(item)
            
    # Return the list with all redundant values removed.
    return unique_lst

def get_values(data, attr):
    """
    Creates a list of values in the chosen attribut for each record in data,
    prunes out all of the redundant values, and return the list.  
    """
    return unique([record[1][attr] for record in data])

def choose_attribute(data, attributes, fitness):
    """
    Cycles through all the attributes and returns the attribute with the
    highest information gain (or lowest entropy).
    """
    best_gain = 0.0
    best_attr = None

    for attr in attributes:
        gain = fitness(data, attr)
        if gain >= best_gain:
            best_gain = gain
            best_attr = attr
                
    return best_attr

def get_examples(data, attr, value):
    """
    Returns a list of all the records in <data> with the value of <attr>
    matching the given value.
    """
    rtn_lst = []
    if not data:
        return rtn_lst
    else:
        for record in data:
            if record[1][attr]==value:
                rtn_lst.append(record)
    return rtn_lst

def get_classification(record, tree):
    """
    This function recursively traverses the decision tree and returns a
    classification for the given record.
    """
    # If the current node is a string, then we've reached a leaf node and
    # we can return it as our answer
    if type(tree) == type("string"):
        return tree

    # Traverse the tree further until a leaf node is found.
    else:
        attr = tree.keys()[0]
        t = tree[attr][record[attr]]
        return get_classification(record, t)

def classify(tree, data):
    """
    Returns a list of classifications for each of the records in the data
    list as determined by the given decision tree.
    """

    classification = []
    
    for record in data:
        classification.append(get_classification(record, tree))

    return classification

def create_decision_tree(data, attributes, fitness_func):
    """
    Returns a new decision tree based on the examples given.
    """
    vals = [x[0] for x in data]
    default = most_frequent(vals)

    # If the dataset is empty or the attributes list is empty, return the
    # default value. When checking the attributes list for emptiness, we
    # need to subtract 1 to account for the target attribute.
    if not data or (len(attributes) - 1) <= 0:
        return default
    # If all the records in the dataset have the same classification,
    # return that classification.
    elif vals.count(vals[0]) == len(vals):
        return vals[0]
    else:
        # Choose the next best attribute to best classify our data
        best = choose_attribute(data, attributes,
                                fitness_func)

        # Create a new decision tree/node with the best attribute and an empty
        # dictionary object--we'll fill that up next.
        tree = {best:{}}

        # Create a new decision tree/sub-node for each of the values in the
        # best attribute field
        for val in get_values(data, best):
            # Create a subtree for the current value under the "best" field
            subtree = create_decision_tree(
                get_examples(data, best, val),
                [attr for attr in attributes if attr != best],
                fitness_func)

            # Add the new subtree to the empty dictionary object in our new
            # tree/node we just created.
            tree[best][val] = subtree

    return tree


In [66]:
import math

def entropy(data):
    """
    Calculates the entropy of the given data set for the target attribute.
    """
    val_freq = {}
    data_entropy = 0.0

    # Calculate the frequency of each of the values in the target attr
    for record in data:
        if (val_freq.has_key(record[0])):
            val_freq[record[0]] += 1.0
        else:
            val_freq[record[0]] = 1.0

    # Calculate the entropy of the data for the target attribute
    for freq in val_freq.values():
        data_entropy += (-freq/len(data)) * math.log(freq/len(data), 2) 
        
    return data_entropy
    
def gain(data, attr):
    """
    Calculates the information gain (reduction in entropy) that would
    result by splitting the data on the chosen attribute (attr).
    """
    val_freq = {}
    subset_entropy = 0.0

    # Calculate the frequency of each of the values in the target attribute
    for record in data:
        if (val_freq.has_key(record[1][attr])):
            val_freq[record[1][attr]] += 1.0
        else:
            val_freq[record[1][attr]] = 1.0

    # Calculate the sum of the entropy for each subset of records weighted
    # by their probability of occuring in the training set.
    for val in val_freq.keys():
        val_prob = val_freq[val] / sum(val_freq.values())
        data_subset = [record for record in data if record[1][attr] == val]
        subset_entropy += val_prob * entropy(data_subset)

    # Subtract the entropy of the chosen attribute from the entropy of the
    # whole data set with respect to the target attribute (and return it)
    return (entropy(data) - subset_entropy)
            


Support function to take care of continuous variables.

In [34]:
##Loading sample data
sample2=[(1,[1,1,30,27,1]),
         (0,[0,0,5,8,1]),
         (1,[1,1,27,16,2]),
         (0,[1,1,2,4,1]),
         (0,[0,0,23,8,0]),
         (2,[1,0,20,18,1]),
         (2,[2,0,15,11,1]),
         (2,[1,2,11,25,1]),
         (0,[0,1,11,23,1]),
         (1,[1,2,8,21,2]),
         (0,[1,1,9,9,1]),
         (0,[0,1,18,2,0]),
         (2,[2,2,7,18,1]),
         (2,[2,1,13,6,1]),
         (2,[2,2,19,8,1])]

In [35]:
from bisect import bisect
def discrete_val(data,ranges):
    k = bisect(ranges,float(data))
    if k == 0:
        return str(ranges[k])+"<"
    elif k == len(ranges):
        return str(ranges[k-1])+">"
    else:
        return str(ranges[k-1])+ "-" + str(ranges[k])  

In [36]:
#Code written according to your input - Not optimized
import copy
def discretize(data,column_ids,n_bins=10):
    new_data = copy.deepcopy(data)
    for column in column_ids:
        k = [row[1][column] for row in new_data]
        col_max = max(k);
        col_min = min(k);
        ranges = list(np.linspace(col_min, col_max, n_bins))
        for idx in range(len(data)):
            new_data[idx][1][column] = discrete_val(data[idx][1][column],ranges)
    return new_data

Sample Output

In [38]:
discretize(sample2,[2,3])

[(1, [1, 1, '30.0>', '27.0>', 1]),
 (0, [0, 0, '2.0-5.11111111111', '7.55555555556-10.3333333333', 1]),
 (1, [1, 1, '26.8888888889-30.0', '15.8888888889-18.6666666667', 2]),
 (0, [1, 1, '2.0-5.11111111111', '2.0-4.77777777778', 1]),
 (0, [0, 0, '20.6666666667-23.7777777778', '7.55555555556-10.3333333333', 0]),
 (2, [1, 0, '17.5555555556-20.6666666667', '15.8888888889-18.6666666667', 1]),
 (2, [2, 0, '14.4444444444-17.5555555556', '10.3333333333-13.1111111111', 1]),
 (2, [1, 2, '8.22222222222-11.3333333333', '24.2222222222-27.0', 1]),
 (0, [0, 1, '8.22222222222-11.3333333333', '21.4444444444-24.2222222222', 1]),
 (1, [1, 2, '5.11111111111-8.22222222222', '18.6666666667-21.4444444444', 2]),
 (0, [1, 1, '8.22222222222-11.3333333333', '7.55555555556-10.3333333333', 1]),
 (0, [0, 1, '17.5555555556-20.6666666667', '2.0-4.77777777778', 0]),
 (2, [2, 2, '5.11111111111-8.22222222222', '15.8888888889-18.6666666667', 1]),
 (2, [2, 1, '11.3333333333-14.4444444444', '4.77777777778-7.55555555556', 1

Now we write a function that will help with bootstrapping. 

In [83]:
import random
# method lifted from:http://code.activestate.com/recipes/273085-sample-with-replacement/
def sample_wr(population, k):
    "Chooses k random elements (with replacement) from a population"
    n = len(population)
    _random, _int = random.random, int  # speed hack 
    result = [None] * k
    for i in xrange(k):
        j = _int(_random() * n)
        result[i] = population[j]
    return result

And now we create the array of samples to pass to Spark.

In [89]:
# n is number of trees in our forest
n=5
length=len(sample)
forest=[sample]
# commence bootstrapping
for i in range(1,n):
    # add tree to forest
    forest.append(sample_wr(sample,length))

In [113]:
# m is the number of columns we will be computing on per tree.
m=3
def choose_columns(num_cols):
    return random.sample(range(num_cols),m)

Finally, we parallelize the forest creation.

In [119]:
#parallelize
forest_rdd=sc.parallelize(forest)
forest_rdd.map(lambda s: create_decision_tree(s,choose_columns(4),gain)).collect()

[{0: {0: 0, 1: {1: {0: 2, 1: 1, 2: 2}}, 2: 2}},
 {0: {0: 0, 1: {2: {0: 1, 1: 2, 2: 2}}, 2: 2}},
 {0: {0: 0, 1: {1: {0: 2, 1: 1, 2: 2}}, 2: 2}},
 {0: {0: 0, 1: {3: {1: 2, 2: 1}}, 2: 2}},
 {2: {0: {1: {1: 0, 2: 1}}, 1: {0: {0: 0, 1: 2, 2: 2}}, 2: 2}}]